In [1]:
import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText
from apache_beam.options.pipeline_options import PipelineOptions, SetupOptions

import numpy as np
import tensorflow as tf

pipeline_options = PipelineOptions()
pipeline_options.view_as(SetupOptions).save_main_session = True

In [2]:
p = beam.Pipeline(options=pipeline_options)

infile = '../notes/wordcount-example.md'
outfile = '../notes/wordcount-example-OUT.md'

lines = p | beam.io.ReadFromText(infile)

out_lines = lines | beam.io.WriteToText(outfile)

p.run()

# tfx

In [3]:
from tfx.utils import types

In [4]:
import collections

In [5]:
isinstance([], collections.Iterable)

True

# MNIST - read byte files

In [1]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import extract_images, extract_labels
from tensorflow.python.platform import gfile

In [2]:
# extract_images

In [3]:
import sys
sys.path.append('..')

In [4]:
from apache_beam_examples import read_write_tfrecord

In [5]:
read_write_tfrecord.maybe_download()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [12]:
local_file = '/tmp/data/mnist/val/images.gz'
with gfile.Open(local_file, 'rb') as f:
    train_images = extract_images(f)

Extracting /tmp/data/mnist/val/images.gz


In [13]:
type(train_images), train_images.shape

(numpy.ndarray, (10000, 28, 28, 1))

In [14]:
local_file = '/tmp/data/train-labels-idx1-ubyte.gz'
with gfile.Open(local_file, 'rb') as f:
    train_labels = extract_labels(f)

Extracting /tmp/data/train-labels-idx1-ubyte.gz


NotFoundError: /tmp/data/train-labels-idx1-ubyte.gz; No such file or directory

In [ ]:
type(train_labels), train_labels.shape

In [ ]:
rows = train_images.shape[1]
cols = train_images.shape[2]
depth = train_images.shape[3]

rows, cols, depth

In [ ]:
index = 0
label = train_labels[index]
type(label), label

In [ ]:
image = train_images[0]
type(image)

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _dict_to_example(data):
    feature = {}
    for k,v in data.items():
        if isinstance(v, np.ndarray):
            feature[k] = _bytes_feature(v.tostring())
        elif isinstance(v, np.uint8):
            feature[k] = _int64_feature(int(v))
        elif isinstance(v, int):
            feature[k] = _int64_feature(v)
    return tf.train.Example(features=tf.train.Features(feature=feature))

data = {
  'height': rows,
  'width': cols,
  'depth': depth,
  'label': label,
  'image_raw': image
}

example = _dict_to_example(data)

example